In [1]:
import requests
import time
import json
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
import threading
import queue
import random
import datetime

from urllib.error import URLError, HTTPError
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from pprint import pprint
from selenium.webdriver.common.alert import Alert
from config.site_config import *

In [2]:
DRIVER_PATH = './config/chromedriver'
CHROME_OPTIONS = Options()

#driver.implicitly_wait(3)

In [3]:
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.maximize_window()
driver.get(saramin_dict['url_main'])

C:\Users\wnsdh\AppData\Local\Temp\ipykernel_14912\3225127078.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


In [4]:
# 로그인
driver.find_element_by_xpath(saramin_dict['cursor']).click()                          # 커서 활성화

AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'

In [4]:

driver.find_element_by_name(saramin_dict['tag_id']).send_keys(saramin_dict['my_id'])  # ID
driver.find_element_by_name(saramin_dict['tag_pw']).send_keys(saramin_dict['my_pw'])  # PW
driver.find_element_by_xpath(saramin_dict['click_login']).click()                     # 로그인 클릭


AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'

In [ ]:
# 이력서 페이지 이동, 수정, 저장
driver.get(saramin_dict['resume_page'])
driver.find_element_by_xpath(saramin_dict['edit_page']).click()
driver.find_element_by_xpath(saramin_dict['save']).click()

# 종료
time.sleep(3)
driver.close()

In [ ]:
https://www.saramin.co.kr/zf_user/search/recruit?searchType=search&searchword=개발자&loc_mcd=101000,102000&company_cd=0,1,2,3,4,5,6,7&panel_type=&search_optional_item=y&search_done=y&panel_count=y&recruitPage=1&recruitSort=relation&recruitPageCount=50&inner_com_type=&show_applied=&quick_apply=&except_read=&ai_head_hunting=&mainSearch=n

searchword=검색어
loc_mcd=지역
company_cd=채용제공구분 / 9(파견/대행), 10(헤드헌팅)
recruitPageCount = 페이지 뷰 수
loc_mcd=지역

searchword=개발자
loc_mcd=101000,102000 (서울,경기)
company_cd=0,1,2,3,4,5,6,7
recruitPageCount=50
loc_mcd=101000,102000